In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import norm
import math

In [2]:
rates = pd.read_csv('./InterestRates.csv')
#rates['key'] = rates['pricedate'].str.cat(rates['maturity'] , sep =", ")
rates['pricedate_str'] = rates['pricedate']
rates['pricedate'] = pd.to_datetime(rates.pricedate, format='%Y-%m-%d', errors='ignore')
#print (rates.dtypes)
#rates.info()
rates[['number1','month']] = rates.maturity.str.split("M",expand=True,)
rates[['number2','year']] = rates.maturity.str.split("Y",expand=True,)

#So, I want to place number with pfv where pfv is not null, how can I achieve that?
rates['number'] = np.where(~rates['year'].isnull(),rates['number2'],rates['number1'])
rates['number'] = pd.to_numeric(rates['number'])
rates['MorY'] = np.where(~rates['year'].isnull(),'Y','M')

rates = rates.drop(columns=['number1','number2','month','year'])
rates['months'] = np.where(rates['MorY']=='Y',rates['number']*12,rates['number']*1)

#generating key2
#total_rows['ColumnID'] = total_rows['ColumnID'].astype(str)
rates['months_str'] = rates['months'].astype(str)
rates['key2'] = rates['pricedate_str'].str.cat(rates['months_str'], sep=',')

rates.info()
rates.head()

#for the risk free rates I am thinking on using the smallest one, 1 month
#rates_1m = rates[rates['maturity']=='1M']
#rates_1m.info()
#rates_1m.head()

#len(rates.key.unique())
#len(rates.maturity.unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7085 entries, 0 to 7084
Data columns (total 9 columns):
pricedate        7085 non-null datetime64[ns]
maturity         7085 non-null object
bidrate          7085 non-null float64
pricedate_str    7085 non-null object
number           7085 non-null int64
MorY             7085 non-null object
months           7085 non-null int64
months_str       7085 non-null object
key2             7085 non-null object
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 498.2+ KB


,pricedate,maturity,bidrate,pricedate_str,number,MorY,months,months_str,key2
0,2020-03-02,100M,1.463333,2020-03-02,100,M,100,100,"2020-03-02,100"
1,2020-03-02,101M,1.466667,2020-03-02,101,M,101,101,"2020-03-02,101"
2,2020-03-02,102M,1.470001,2020-03-02,102,M,102,102,"2020-03-02,102"
3,2020-03-02,103M,1.478335,2020-03-02,103,M,103,103,"2020-03-02,103"
4,2020-03-02,104M,1.486669,2020-03-02,104,M,104,104,"2020-03-02,104"


In [3]:
prices = pd.read_csv('./PricesFile1.csv')
#drop other variables
prices = prices[prices['priceindex']=='AECO NIT MONTHLY (7A) US$ FWD']
#'AECO NIT MONTHLY (7A) US'

#Gen Key
prices['pricedate_str'] = prices['pricedate']
prices['delivdate_str'] = prices['delivdate']
prices['key'] = prices['pricedate_str'].str.cat(prices['delivdate_str'], sep=',')

#fixing dates
#pricedate and delivdate
prices['pricedate'] = pd.to_datetime(prices.pricedate, format='%Y-%m-%d', errors='ignore')
prices['delivdate'] = pd.to_datetime(prices.delivdate, format='%Y-%m-%d', errors='ignore')
#prices.info()
#negative prices
neg_prices = prices[prices['price']<0]

#calculating duration ****QUESTION Should we drop negatives according to scott answer?
prices['duration'] = prices['delivdate'] - prices['pricedate']
prices['duration_months'] = round(prices['duration'].dt.days/30).astype(int)

#generating key2
prices['duration_months'][prices['duration_months']==0] = 1
prices['duration_months_str'] = prices['duration_months'].astype(str)
prices['key2'] = prices['pricedate_str'].str.cat(prices['duration_months_str'], sep=',')

#prices.info()

#drop negative duration  *0 elements drop
prices_drop = prices[prices['duration'].dt.days<0]
prices = prices[prices['duration'].dt.days>=0]

prices = prices.drop(columns=['pricedate_str','delivdate_str'])

#prices[(prices['pricedate']=='2020-04-20')& (prices['delivdate']=='2020-05-01')]
prices.info()
#Day of negative
#prices[prices['pricedate']=='2020-04-20']

#Checking if prices are unique for same dates
#len(prices.key.unique())

#prices.priceindex.unique()
#neg_prices.info()
#neg_prices.head()
#prices.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2049 entries, 3 to 2051
Data columns (total 9 columns):
priceindex             2049 non-null object
pricedate              2049 non-null datetime64[ns]
delivdate              2049 non-null datetime64[ns]
price                  2049 non-null float64
key                    2049 non-null object
duration               2049 non-null timedelta64[ns]
duration_months        2049 non-null int64
duration_months_str    2049 non-null object
key2                   2049 non-null object
dtypes: datetime64[ns](2), float64(1), int64(1), object(4), timedelta64[ns](1)
memory usage: 160.1+ KB


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
prices_NG = pd.read_csv('./PricesFile1.csv')
prices_NG['pricedate_str'] = prices_NG['pricedate']
prices_NG['delivdate_str'] = prices_NG['delivdate']
prices_NG['key'] = prices_NG['pricedate_str'].str.cat(prices_NG['delivdate_str'], sep=',')

#drop other variables
prices_NG = prices_NG[prices_NG['priceindex']=='NYMEX Natural Gas']
#'AECO NIT MONTHLY (7A) US$' 'AECO NIT MONTHLY (7A) US$ FWD'

#fixing dates
#pricedate and delivdate
prices_NG['pricedate'] = pd.to_datetime(prices_NG.pricedate, format='%Y-%m-%d', errors='ignore')
prices_NG['delivdate'] = pd.to_datetime(prices_NG.delivdate, format='%Y-%m-%d', errors='ignore')
#prices_NG.info()
#negative prices_NG
neg_prices_NG = prices_NG[prices_NG['price']<0]


#calculating duration ****QUESTION Should we drop negatives according to scott answer?
prices_NG['duration'] = prices_NG['delivdate'] - prices_NG['pricedate']
prices_NG['duration_months'] = round(prices_NG['duration'].dt.days/30).astype(int)

#generating key2
prices_NG['duration_months'][prices_NG['duration_months']==0] = 1
prices_NG['duration_months_str'] = prices_NG['duration_months'].astype(str)
prices_NG['key2'] = prices_NG['pricedate_str'].str.cat(prices_NG['duration_months_str'], sep=',')

#prices_NG.info()

#drop negative duration  **63 elements dropped
prices_NG_drop = prices_NG[prices_NG['duration'].dt.days<0]
prices_NG = prices_NG[prices_NG['duration'].dt.days>=0]

prices_NG = prices_NG.drop(columns=['pricedate_str','delivdate_str'])

prices_NG.head()
#prices_NG[(prices_NG['pricedate']=='2020-04-20')& (prices_NG['delivdate']=='2020-05-01')]

#Day of negative
#prices_NG[prices_NG['pricedate']=='2020-04-20']

#Checking if prices_NG are unique for same dates
#len(prices_NG.key.unique())

#prices_NG.priceindex.unique()
#neg_prices_NG.info()
#neg_prices_NG.head()
#prices_NG.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,priceindex,pricedate,delivdate,price,key,duration,duration_months,duration_months_str,key2
2052,NYMEX Natural Gas,2020-03-02,2020-04-01,1.755,"2020-03-02,2020-04-01",30 days,1,1,"2020-03-02,1"
2053,NYMEX Natural Gas,2020-03-02,2020-05-01,1.796,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2"
2054,NYMEX Natural Gas,2020-03-02,2020-06-01,1.857,"2020-03-02,2020-06-01",91 days,3,3,"2020-03-02,3"
2055,NYMEX Natural Gas,2020-03-02,2020-07-01,1.934,"2020-03-02,2020-07-01",121 days,4,4,"2020-03-02,4"
2056,NYMEX Natural Gas,2020-03-02,2020-08-01,1.968,"2020-03-02,2020-08-01",152 days,5,5,"2020-03-02,5"


In [5]:
AECO_imvl20=pd.read_csv("./NG_ImpliedVols2020.csv")
AECO_imvl21=pd.read_csv("./NG_ImpliedVols2021.csv")
AECO_imvl22=pd.read_csv("./NG_ImpliedVols2022.csv")

#Append
frames=[AECO_imvl20,AECO_imvl21,AECO_imvl22]
AECO_imvl = pd.concat(frames)

#generatiAECO key
AECO_imvl['volatilitydate_str'] = AECO_imvl['volatilitydate']
AECO_imvl['begtime_str'] = AECO_imvl['begtime']
AECO_imvl['key'] = AECO_imvl['volatilitydate_str'].str.cat(AECO_imvl['begtime_str'], sep=',')

#fixiAECO dates
#volatilitydate and begtime
AECO_imvl['volatilitydate'] = pd.to_datetime(AECO_imvl.volatilitydate, format='%Y-%m-%d', errors='ignore')
AECO_imvl['begtime'] = pd.to_datetime(AECO_imvl.begtime, format='%Y-%m-%d', errors='ignore')

#calculatiAECO duration ****QUESTION Should we drop negatives accordiAECO to scott answer?
AECO_imvl['duration'] = AECO_imvl['begtime'] - AECO_imvl['volatilitydate']
AECO_imvl['duration_months'] = round(AECO_imvl['duration'].dt.days/30,0).astype(int)

#Take previous day volatility
#AECO_imvl.info()

#drop negative duration  **No elements dropped
AECO_imvl = AECO_imvl[AECO_imvl['duration_months']>=0]
#CheckiAECO if AECO_imvl are unique for same dates
len(AECO_imvl.key.unique())

###generatiAECO key2
#AECO_imvl['duration_months'][AECO_imvl['duration_months']==0] = 1
#AECO_imvl['duration_months_str'] = AECO_imvl['duration_months'].astype(str)
#AECO_imvl['key2'] = AECO_imvl['volatilitydate_str'].str.cat(AECO_imvl['duration_months_str'], sep=',')




1936

In [6]:
####To calculate (F-K)/K
NG_imvl = AECO_imvl

NG_imvl = NG_imvl.drop(columns=['volatilitydate','begtime','duration','duration_months','volatilitydate_str','begtime_str'])

#Left join
master_NG = pd.merge(prices_NG,NG_imvl,on='key',how='left')

#Replace NA's by the previous observation 
na_master_NG = master_NG[np.isnan(master_NG['volatility'])]
na_master_NG.reset_index(drop=True, inplace=True)

for i in range(len(na_master_NG)):
    first_date = na_master_NG['pricedate'][i]
    last_date = na_master_NG['delivdate'][i]
    sub_set = master_NG[(master_NG['pricedate']<=first_date)& (master_NG['delivdate']==last_date)].sort_values(by='pricedate', ascending=False)
    sub_set.reset_index(drop=True, inplace=True)
    j=0
    while np.isnan(sub_set['volatility'][j]) and sub_set['pricedate'][j] > datetime.strptime('2020-03-02', '%Y-%m-%d'):
        j = j+1    # update counter
    
    if j>0 and sub_set['pricedate'][j] > datetime.strptime('2020-03-02', '%Y-%m-%d'):
        index_save = master_NG.index[(master_NG['pricedate']==first_date)& (master_NG['delivdate']==last_date)][0]
        master_NG['volatilityindex'].at[index_save]=(sub_set['volatilityindex'][j+1])
        master_NG['strikeprice'].at[index_save]=(sub_set['strikeprice'][j+1])
        master_NG['volatility'].at[index_save]=(sub_set['volatility'][j+1])
        #master_NG['key2'].at[index_save]=(sub_set['key2'][j+1])
        del(index_save)

    del(first_date,last_date,sub_set)


#master_NG.info()
master_NG = master_NG.dropna()

#Lets calculate the (F-K)/K
master_NG['moneyness'] = (master_NG['price']-master_NG['strikeprice'])/(master_NG['strikeprice'])

#master_AECO[(master_AECO['pricedate']=='2020-04-20')& (master_AECO['delivdate']=='2020-05-01')]
            
#len(master_AECO[np.isnan(master_AECO['volatility'])])

master_NG.info()
master_NG.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 152833 entries, 0 to 152859
Data columns (total 13 columns):
priceindex             152833 non-null object
pricedate              152833 non-null datetime64[ns]
delivdate              152833 non-null datetime64[ns]
price                  152833 non-null float64
key                    152833 non-null object
duration               152833 non-null timedelta64[ns]
duration_months        152833 non-null int64
duration_months_str    152833 non-null object
key2                   152833 non-null object
volatilityindex        152833 non-null object
strikeprice            152833 non-null float64
volatility             152833 non-null float64
moneyness              152833 non-null float64
dtypes: datetime64[ns](2), float64(4), int64(1), object(5), timedelta64[ns](1)
memory usage: 16.3+ MB


,priceindex,pricedate,delivdate,price,key,duration,duration_months,duration_months_str,key2,volatilityindex,strikeprice,volatility,moneyness
0,NYMEX Natural Gas,2020-03-02,2020-04-01,1.755,"2020-03-02,2020-04-01",30 days,1,1,"2020-03-02,1",V_NYMEX Natural Gas,1.03,0.592,0.703883
1,NYMEX Natural Gas,2020-03-02,2020-04-01,1.755,"2020-03-02,2020-04-01",30 days,1,1,"2020-03-02,1",V_NYMEX Natural Gas,1.08,0.576,0.625000
2,NYMEX Natural Gas,2020-03-02,2020-04-01,1.755,"2020-03-02,2020-04-01",30 days,1,1,"2020-03-02,1",V_NYMEX Natural Gas,1.13,0.562,0.553097
3,NYMEX Natural Gas,2020-03-02,2020-04-01,1.755,"2020-03-02,2020-04-01",30 days,1,1,"2020-03-02,1",V_NYMEX Natural Gas,1.18,0.548,0.487288
4,NYMEX Natural Gas,2020-03-02,2020-04-01,1.755,"2020-03-02,2020-04-01",30 days,1,1,"2020-03-02,1",V_NYMEX Natural Gas,1.23,0.535,0.426829


In [7]:
master_NG = master_NG.drop(columns=['priceindex','pricedate','delivdate','price','duration','duration_months','duration_months_str','key2'])

#Left join
master_AECO = pd.merge(prices,master_NG,on='key',how='left')

master_AECO['strikeprice_AECO'] = (master_AECO['price'])/(master_AECO['moneyness'] + 1)


In [8]:
#MergiAECO Rates
rates2 = rates.drop(columns=['pricedate','pricedate_str','number','MorY','months','months_str'])

#Left join
master_AECO = pd.merge(master_AECO,rates2,on='key2',how='left')
#master_AECO[(master_AECO['pricedate']=='2020-04-20')& (master_AECO['delivdate']=='2020-05-01')]

master_AECO.to_csv ('master_AECO_complete.csv', index = False, header=True)
master_AECO.info()
#master_AECO.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152891 entries, 0 to 152890
Data columns (total 16 columns):
priceindex             152891 non-null object
pricedate              152891 non-null datetime64[ns]
delivdate              152891 non-null datetime64[ns]
price                  152891 non-null float64
key                    152891 non-null object
duration               152891 non-null timedelta64[ns]
duration_months        152891 non-null int64
duration_months_str    152891 non-null object
key2                   152891 non-null object
volatilityindex        152833 non-null object
strikeprice            152833 non-null float64
volatility             152833 non-null float64
moneyness              152833 non-null float64
strikeprice_AECO       152833 non-null float64
maturity               152891 non-null object
bidrate                152891 non-null float64
dtypes: datetime64[ns](2), float64(6), int64(1), object(6), timedelta64[ns](1)
memory usage: 19.8+ MB


In [9]:
#Black 76


# t = pricedate          1198 non-null datetime64[ns]
# T = delivdate          1198 non-null datetime64[ns]
# F(t,T) = price              1198 non-null float64
# T - t = duration_n         1198 non-null int64
# K = strikeprice        1198 non-null float64
# sigma = volatility         1198 non-null float64
# r = bidrate            1198 non-null float64

master_AECO.rename(columns={'pricedate': 't', 'delivdate': 'T', 'price': 'F', 'strikeprice_AECO': 'K', 'volatility': 'sigma', 'bidrate': 'r'}, inplace=True)

master_AECO['T_t'] = master_AECO['duration'].dt.days

#FIX A Te
#For creatiAECO the loop, check if Te_t < T, if true, calculate the call value
#Te_t = 15/360

master_AECO = master_AECO[master_AECO['T_t']>30]
Te_t = (master_AECO['T_t']-30)/360

#CalculatiAECO call value
Nd1 = norm.cdf((np.log(master_AECO['F']/master_AECO['K']) + (1/2 * master_AECO['sigma']**2 * Te_t))/(master_AECO['sigma'] * np.sqrt(Te_t)))
Nd2 = norm.cdf((np.log(master_AECO['F']/master_AECO['K']) - (1/2 * master_AECO['sigma']**2 * Te_t))/(master_AECO['sigma'] * np.sqrt(Te_t)))

master_AECO['call'] = np.exp(-(master_AECO['r']/100)*Te_t)*(master_AECO['F']*Nd1 - master_AECO['K']*Nd2)

Nd1_put = norm.cdf(-(np.log(master_AECO['F']/master_AECO['K']) + (1/2 * master_AECO['sigma']**2 * Te_t))/(master_AECO['sigma'] * np.sqrt(Te_t)))
Nd2_put = norm.cdf(-(np.log(master_AECO['F']/master_AECO['K']) - (1/2 * master_AECO['sigma']**2 * Te_t))/(master_AECO['sigma'] * np.sqrt(Te_t)))

master_AECO['put'] = np.exp(-(master_AECO['r']/100)*Te_t)*(master_AECO['K']*Nd2_put - master_AECO['F']*Nd1_put )

#master_AECO[(master_AECO['t']=='2020-04-20')& (master_AECO['T']=='2020-05-01')]
master_AECO.info()
#master_AECO.head()

/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 149493 entries, 49 to 152890
Data columns (total 19 columns):
priceindex             149493 non-null object
t                      149493 non-null datetime64[ns]
T                      149493 non-null datetime64[ns]
F                      149493 non-null float64
key                    149493 non-null object
duration               149493 non-null timedelta64[ns]
duration_months        149493 non-null int64
duration_months_str    149493 non-null object
key2                   149493 non-null object
volatilityindex        149436 non-null object
strikeprice            149436 non-null float64
sigma                  149436 non-null float64
moneyness              149436 non-null float64
K                      149436 non-null float64
maturity               149493 non-null object
r                      149493 non-null float64
T_t                    149493 non-null int64
call                   149436 non-null float64
put                    149436 

In [10]:
master_AECO.to_csv ('master_AECO_call_put_black76.csv', index = False, header=True)
master_AECO.head()

,priceindex,t,T,F,key,duration,duration_months,duration_months_str,key2,volatilityindex,strikeprice,sigma,moneyness,K,maturity,r,T_t,call,put
49,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-05-01,-0.616,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_NYMEX Natural Gas,1.01,0.605,0.778218,-0.346414,2M,0.995003,60,-0.269373,-0.000010
50,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-05-01,-0.616,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_NYMEX Natural Gas,1.06,0.590,0.694340,-0.363563,2M,0.995003,60,-0.252249,-0.000022
51,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-05-01,-0.616,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_NYMEX Natural Gas,1.11,0.575,0.618018,-0.380713,2M,0.995003,60,-0.235136,-0.000044
52,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-05-01,-0.616,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_NYMEX Natural Gas,1.15,0.609,0.561739,-0.394432,2M,0.995003,60,-0.221539,-0.000155
53,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-05-01,-0.616,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_NYMEX Natural Gas,1.20,0.589,0.496667,-0.411581,2M,0.995003,60,-0.204502,-0.000253
